# Spark Preparation
We check if we are in Google Colab.  If this is the case, install all necessary packages.

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.3.2 with hadoop 3.3, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab.
Learn more from [A Must-Read Guide on How to Work with PySpark on Google Colab for Data Scientists!](https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/)

In [3]:
!pip install pyspark

In [4]:
try:
    import google.colab

    IN_COLAB = True
except:
    IN_COLAB = False

In [5]:
if IN_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget -q https://archive.apache.org/dist/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
    !tar xf spark-3.3.2-bin-hadoop3.tgz
    !mv spark-3.3.2-bin-hadoop3 spark
    !pip install -q findspark
    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark"

# Start a Local Cluster

In [6]:
import findspark

findspark.init()

import pyspark
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *

spark = SparkSession.builder.master("local[*]").appName("pyspark_colab").getOrCreate()

# Spark Assignment

Based on the movie review dataset in 'netflix-rotten-tomatoes-metacritic-imdb.csv', answer the below questions.

**Note:** do not clean or remove missing data

In [8]:
path = "netflix-rotten-tomatoes-metacritic-imdb.csv"
df = spark.read.csv(path, header=True, inferSchema=True)
df.createOrReplaceTempView("netflix")

In [9]:
df.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Tags: string (nullable = true)
 |-- Languages: string (nullable = true)
 |-- Series or Movie: string (nullable = true)
 |-- Hidden Gem Score: double (nullable = true)
 |-- Country Availability: string (nullable = true)
 |-- Runtime: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Writer: string (nullable = true)
 |-- Actors: string (nullable = true)
 |-- View Rating: string (nullable = true)
 |-- IMDb Score: string (nullable = true)
 |-- Rotten Tomatoes Score: string (nullable = true)
 |-- Metacritic Score: string (nullable = true)
 |-- Awards Received: double (nullable = true)
 |-- Awards Nominated For: double (nullable = true)
 |-- Boxoffice: string (nullable = true)
 |-- Release Date: string (nullable = true)
 |-- Netflix Release Date: string (nullable = true)
 |-- Production House: string (nullable = true)
 |-- Netflix Link: string (nullable = true)
 |-- IMDb Link: string (null

#### What is the maximum and average of the overall hidden gem score?

In [ ]:
df.select(max("Hidden Gem Score"), avg("Hidden Gem Score")).show()

In [11]:
cmd = """
SELECT MAX(`Hidden Gem Score`), AVG(`Hidden Gem Score`)
FROM netflix
"""
spark.sql(cmd).show()

+---------------------+---------------------+
|max(Hidden Gem Score)|avg(Hidden Gem Score)|
+---------------------+---------------------+
|                  9.8|    5.937551386501234|
+---------------------+---------------------+



#### How many movies that are available in Korea?

In [12]:
df.filter(df["Languages"].contains("Korean")).count()

735

In [13]:
cmd = """
SELECT COUNT(*) as count
FROM netflix
WHERE Languages LIKE '%Korean%'
"""
spark.sql(cmd).show()

+-----+
|count|
+-----+
|  735|
+-----+



#### Which director has the highest average hidden gem score?

In [ ]:
df.groupby("Director").agg({"Hidden Gem Score": "avg"}).withColumnRenamed(
    "avg(Hidden Gem Score)", "avg"
).orderBy(col("avg").desc()).limit(5).show()

In [15]:
cmd = """
SELECT Director, AVG(`Hidden Gem Score`) as avg
FROM netflix
GROUP BY Director
ORDER BY avg DESC
LIMIT 5
"""
spark.sql(cmd).show()

+--------------------+---+
|            Director|avg|
+--------------------+---+
|         Dorin Marcu|9.8|
|    Fernando Escovar|9.6|
|          Rosa Russo|9.5|
|         Kate Brooks|9.5|
|Vincent Bal, Kenn...|9.5|
+--------------------+---+



#### How many genres are there in the dataset?

In [16]:
# split: str -> list
# explode: list -> rows
# alias: default name col -> genre
# trim row str
# .show(X): default 20
# .collect(): save in var
# .count(): countdd

df.select(explode(split("Genre", ",")).alias("genre")).withColumn(
    "genre", trim(col("genre"))
).distinct().count()

28

In [17]:
spark.stop()